![](http://)## Jane Street: NN Standard

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os, gc
import pandas as pd
import numpy as np
import janestreet
import datatable as dt

from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import GroupKFold
# from sklearn.model_selection import GroupTimeSeriesSplit 
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from joblib import dump, load


import tensorflow as tf
device = 'cuda' if tf.config.list_physical_devices('GPU') else 'cpu'
print(device)
from tensorflow import keras

tf.random.set_seed(42)
import tensorflow.keras.backend as K
import tensorflow.keras.layers as layers
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

if device == 'cuda':
    import cudf
    import cupy as cp

## Processing data

In [ ]:
%%time
print('Loading...')
train = dt.fread('/kaggle/input/jane-street-market-prediction/train.csv')

print('Converting...')
train = train.to_pandas()
print('Finish.')

In [ ]:
nulls = train.isnull().sum()
nulls_list = list(nulls[nulls >(0.1 * len(train))].index)
nulls_list

In [ ]:
# ignore_columns = ['resp_1', 'resp_2', 'resp_3', 'resp_4', 'resp','ts_id','target']
features = [c for c in train.columns if 'feature' in c and c not in nulls_list]
features

In [ ]:
%%time
# Filling the missing data 
print('Filling...')
f_mean = train[features[1:]].mean()
train = train.query('weight > 0').reset_index(drop = True)
train[features[1:]] = train[features[1:]].fillna(f_mean)
train['action'] = (train['resp'] > 0).astype('int')

f_mean = np.array(f_mean)
np.save('f_mean.npy', f_mean)

print('Finish.')

In [ ]:
train.sort_values(by= ['date','ts_id'],inplace=True)


In [ ]:
# reduce memory
def reduce_memory_usage(df):
    
    start_memory = df.memory_usage().sum() / 1024**2
    print(f"Memory usage of dataframe is {start_memory} MB")
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != 'object':
            c_min = df[col].min()
            c_max = df[col].max()
            
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    pass
        else:
            df[col] = df[col].astype('category')
    
    end_memory = df.memory_usage().sum() / 1024**2
    print(f"Memory usage of dataframe after reduction {end_memory} MB")
    print(f"Reduced by {100 * (start_memory - end_memory) / start_memory} % ")
    return df

In [ ]:
%%time
train = reduce_memory_usage(train)

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
train.info()

In [ ]:
def create_mlp(num_columns, num_labels,label_smoothing,learning_rate):
    
    # defining a model applying dropout regularization
    model = keras.models.Sequential([
    keras.layers.Input(shape = (num_columns, )),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(1000, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(1000, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(1, activation="sigmoid")
    ])
    
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate),
                  loss = tf.keras.losses.BinaryCrossentropy(label_smoothing = label_smoothing), 
                  metrics = tf.keras.metrics.AUC(name = 'AUC'), 
                 )
    
    return model

In [ ]:
batch_size = 20000
hidden_units = [384, 896, 896, 394]
dropout_rates = [0.10143786981358652, 0.19720339053599725, 0.1123435323 ,0.23148340929571917, 0.2157768967777311]
label_smoothing = 1e-2
learning_rate = 1e-3

In [ ]:
model = create_mlp(len(features),1,label_smoothing,learning_rate)

In [ ]:
model.summary()

In [ ]:
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class GroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_size : int, default=None
        Maximum size for a single training set.
    Examples
    --------
    >>> import numpy as np
    >>> from sklearn.model_selection import GroupTimeSeriesSplit
    >>> groups = np.array(['a', 'a', 'a', 'a', 'a', 'a',\
                           'b', 'b', 'b', 'b', 'b',\
                           'c', 'c', 'c', 'c',\
                           'd', 'd', 'd'])
    >>> gtss = GroupTimeSeriesSplit(n_splits=3)
    >>> for train_idx, test_idx in gtss.split(groups, groups=groups):
    ...     print("TRAIN:", train_idx, "TEST:", test_idx)
    ...     print("TRAIN GROUP:", groups[train_idx],\
                  "TEST GROUP:", groups[test_idx])
    TRAIN: [0, 1, 2, 3, 4, 5] TEST: [6, 7, 8, 9, 10]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a']\
    TEST GROUP: ['b' 'b' 'b' 'b' 'b']
    TRAIN: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10] TEST: [11, 12, 13, 14]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a' 'b' 'b' 'b' 'b' 'b']\
    TEST GROUP: ['c' 'c' 'c' 'c']
    TRAIN: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]\
    TEST: [15, 16, 17]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a' 'b' 'b' 'b' 'b' 'b' 'c' 'c' 'c' 'c']\
    TEST GROUP: ['d' 'd' 'd']
    """
    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_size=None
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_size = max_train_size

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))
        group_test_size = n_groups // n_folds
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []
            for train_group_idx in unique_groups[:group_test_start]:
                train_array_tmp = group_dict[train_group_idx]
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)
            train_end = train_array.size
            if self.max_train_size and self.max_train_size < train_end:
                train_array = train_array[train_end -
                                          self.max_train_size:train_end]
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)
            yield [int(i) for i in train_array], [int(i) for i in test_array]

## Training

In [ ]:
%%time

print ("Training Started .......................")

oof = np.zeros(len(train['action']))
# gkf = GroupTimeSeriesSplit(n_splits = 5)
gkf = GroupKFold(n_splits = 5)

val_idx = []
oof_scores = []

for fold, (tr, te) in enumerate(gkf.split(train[features].values, train['action'].values, train['date'].values)):
    
#     print("#"*75)
#     print(f"Fold{fold}")
    
    X_tr, X_val = train.loc[tr, features].values, train.loc[te, features].values
    y_tr, y_val = train.loc[tr, 'action'].values, train.loc[te, 'action'].values
    
#     print(f"train split shape is {X_tr.shape}, validation split shape is {X_val.shape}")
#     print('Training...')
    
    ckp_path = f'JS_NN_Model_{fold}.hdf5'
    model = create_mlp(X_tr.shape[1],1,label_smoothing,learning_rate)
    rlr = ReduceLROnPlateau(monitor = 'val_AUC', factor = 0.1, patience = 3, verbose = 0, 
                            min_delta = 1e-4, mode = 'max')
    ckp = ModelCheckpoint(ckp_path, monitor = 'val_AUC', verbose = 0, 
                          save_best_only = True, save_weights_only = True, mode = 'max')
    es = EarlyStopping(monitor = 'val_AUC', min_delta = 1e-4, patience = 7, mode = 'max', 
                       baseline = None, restore_best_weights = True, verbose = 0)
    model.fit(X_tr, y_tr, validation_data = (X_val, y_val), epochs = 100, 
              batch_size = batch_size,callbacks = [rlr, ckp, es],verbose = 2)
    model.save_weights(ckp_path)
    
#     print("Fine tuning...")
  
    # Finetune 5 epochs on validation set with small learning rate
    ckp_path_ft = f'JS_NN_Model_{fold}_finetuning.hdf5'
    model = create_mlp(X_tr.shape[1],1,label_smoothing,learning_rate / 100)
    model.load_weights(ckp_path)
    ckp_ft = ModelCheckpoint(ckp_path_ft, monitor = 'AUC', verbose = 0,
                             save_best_only = True, save_weights_only = True, mode = 'max')
    model.fit(X_val, y_val, epochs = 5, batch_size = batch_size,callbacks = [ckp_ft], verbose = 2)
    model.save_weights(ckp_path_ft)
    
    oof[te] += model.predict(X_val, batch_size = batch_size * 4).ravel()
    val_idx.append(te)
    score = roc_auc_score(y_val, oof[te])
    oof_scores.append(score)
    K.clear_session()
    del model, X_tr, X_val, y_tr, y_val
    rubbish = gc.collect()
    print(f'Fold {fold} ROC AUC:\t', score)
print ("Training Finished .......................")

In [ ]:
max_idx = oof_scores.index(max(oof_scores))
print(f'Max oof score is {max(oof_scores)} in FOLD{oof_scores.index(max(oof_scores))}')

In [ ]:
# indices = np.array([])
# for item in val_idx:
#     indices = np.append(indices,item)
# indices = indices.astype(int)
# score_oof = roc_auc_score(train['action'].values[indices], oof[indices])
# print(score_oof)

## Load Model

Currently, it took a long time to make prediction using the JS env API, so I could only load one model to predict test set.

In [ ]:
!pwd
!ls -lrt


In [ ]:
num_models = 1
models = []
for i in range(num_models):
    clf = create_mlp(len(features),1,label_smoothing,learning_rate)
# clf.load_weights(f'JS_NN_Model_4_finetuning.hdf5')
    clf.load_weights(f'JS_NN_Model_4_finetuning.hdf5')
    models.append(clf)


In [ ]:
models

In [ ]:
f_mean = np.load('f_mean.npy')


## Submitting

In [ ]:
env = janestreet.make_env()
env_iter = env.iter_test()

In [ ]:
opt_th = 0.5
for (test_df, pred_df) in tqdm(env_iter):
    if test_df['weight'].item() > 0:
        x_tt = test_df.loc[:, features].values
        if np.isnan(x_tt[:, 1:].sum()):
            x_tt[:, 1:] = np.nan_to_num(x_tt[:, 1:]) + np.isnan(x_tt[:, 1:]) * f_mean
        pred = 0.
        for clf in models:
            pred += clf(x_tt, training = False).numpy().item() / num_models
#         pred = models[0](x_tt, training = False).numpy().item()
        pred_df.action = np.where(pred >= opt_th, 1, 0).astype(int)
    else:
        pred_df.action = 0
    env.predict(pred_df)